# Mydocstring tutorial
Welcome to the mydocstring tutorial. This tutorial will show you how to parse a function that contains a docstring that has been documented using the Google-style docstring format.

Before we begin you should know that mydocstring is not a complete documentation system. Instead, it contains a few core components that are designed to help you build you a documentation system just the way you want it. In essence, mydocstring parses docstrings and packs documentation for sections, arguments, and so forth into dictoinary. At the end of you day, you have control of the data that makes up a docstring, and you can do whatever you want with it.


## Extracting info from functions and class methods


### Functions
To get started, we need to have some data to work with. Consider the following string that contains source code for a function for adding two numbers. You can modify it if you like.

In [1]:
source = ("""def add(a: float, b: float) -> float:
                \"""
                Add two floating-point numbers together.
    
                Args:
                    a : First float to add.
                    b : Second float to add.
                        
                Returns:
                    The result of the operation.
                \"""
                return a + b
                """)

Note that in the example above, the docstring of the function is encapulsated in """ quotes. This is necessary for the detection of the docstring to work correctly for python source code. An exception is auto-generated docstrings from pybind, which we will explore later. Also, the signature is annotated with argument types, and a return type. That said, annotations are optional.

Now, let's extract information from this function. It is important to pick the right extraction tool for the job. In this case, since we are working with usual python source code, we want to use `PyExtract` (use `PyBindExtract` for pybind docstrings). 

In [2]:
import mydocstring as mydoc

In [3]:
extract = mydoc.extract.PyExtract(source)
add = extract.extract('add')

The return value `add` is a dictionary that contains the function signature, docstring, and so forth. The reason why we need to specify what function to extract is because the source code can contain multiple functions, and maybe only some of them are of interest to us.

Here is the docstring

In [4]:
print(add['docstring'])



Add two floating-point numbers together.

Args:
    a : First float to add.
    b : Second float to add.
        
Returns:
    The result of the operation.



Here are the arguments, their annotations, as well as the return type. 

In [5]:
add['parsed_signature']

{'args': {'a': 'float', 'b': 'float'}, 'return_annotation': 'float'}

If you ever need to parse a signature on its own you can do so as follows.

In [6]:
mydoc.parse.parse_signature('(a: int, b: int, default: bool=True) -> int)')

{'args': {'a': 'int', 'b': 'int', 'default': 'bool=True'},
 'return_annotation': 'int'}

### Class methods
Now, let's look at a class method and how to extract one of those. 


In [7]:
source = ("""
            class Math(object):
                \"""
                Class that contains mathematical operations.
                \"""
                def add(self, a: float, b: float) -> float:
                    \"""
                    Add two floating-point numbers together.
    
                    Args:
                        a : First float to add.
                        b : Second float to add.
                        
                    Returns:
                        The result of the operation.
                    \"""
                    return a + b
            """)

This code snippet contains the `add` function from before, and also a class that contains method with the same name as the `add` function. To signal to the extract function what we want to extract, we use the class name to distinguish between a function and a method.

In [8]:
extract = mydoc.extract.PyExtract(source)
math = extract.extract('Math')
math_add = extract.extract('Math.add') 

In [9]:
math_add['signature']

'(self, a: float, b: float) -> float'

### Pybind
Docstrings generated by pybind look slightly different from the ones in the Python source code. The key difference is that both the function name and the documentation is contained in the same string. To parse these strings, we need to use `PyBindExtract`, like this.

In [10]:
source = """add(a: float, b: float) -> float
            Add two floating-point numbers together.
    
            Args:
                a : First float to add.
                b : Second float to add.
                        
            Returns:
                The result of the operation.
        """
add = mydoc.extract.PyBindExtract(source).extract('add')

Besides the slightly different syntax in the string itself, and the need to use `PyBindExtract` everything else works exactly the same as before. For example, to inspect the parsed signature, we use

In [11]:
add['parsed_signature']

{'args': {'a': 'float', 'b': 'float'}, 'return_annotation': 'float'}

## Parsing docstrings
Let's see how to parse a docstring stored in the google docstring format. We import the `GoogleDocString` parser tool and call its `parse` function. It is a good idea to pass the parsed signature as well because then the docstring parser can warn about any undocumented arguments.

In [12]:
source = ("""def add(a: float, b: float) -> float:
                \"""
                Add two floating-point numbers together.
    
                Args:
                    a : First float to add.
                    b : Second float to add.
                        
                Returns:
                    The result of the operation.
                \"""
                return a + b
                """)
add = mydoc.extract.PyExtract(source).extract('add')
add_doc = mydoc.parse.GoogleDocString(add['docstring'], signature=add['parsed_signature']).parse()

The parser works by splitting a docstring section by section. The example above contains three sections, the first one starting with *Add two...*, then there is section *Args..*, and finally a section *Returns*. Each section starts with an optional header (such as *Args*) and can contain an argument or text.

In [13]:
print(add_doc[0]['text'])



Add two floating-point numbers together.



Here is the argument list found in the second section.

In [14]:
args = add_doc[1]['args']
for arg in args:
    print('%s(%s): %s'% (arg['field'], arg['signature'], arg['description']))

a(float): First float to add.
b(float): Second float to add.
    


Note that the type has been automatically inferred. This was possible because we passed `add['parsed_signature']` during initialization.

Here is the header and text in the last section.

In [15]:
print('%s: %s' %(add_doc[2]['header'], add_doc[2]['text']))

Returns: The result of the operation.

